# Тестовое задание

В данном задании необходимо написать код, который считает следующие признаки для каждого клиента `user_id`:

1. `events_ordinal_number` - порядковый номер события
2. `second_event_time` - время второго события
2.	`loan_ordinal_number` - порядковый номер займа
2.	`events_time_diff` - разница во времени между событиями
2.	`previous_loans_max_amount` - максимальная сумма предыдущего займа.


### Подготовка датафрейма

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
size = 10000

df = pd.DataFrame(
    {
        'time': [*pd.date_range('2001-01-01', '2020-09-01', freq='1h')][:size],
        'user_id': np.random.choice(1000, size),
        'type':    np.random.choice(2, size, p=[0.8, 0.2]),
        'amount': [int(x//1) for x in  np.random.normal(15_000, 1_000, size)],
        'target': np.random.choice(2, size, p=[0.5, 0.5]),
    },
    index=[*range(size)],
)

In [3]:
df['is_loan'] = df['type']
df['type'] = df['type'].replace({0: 'loanRequest', 1: 'loan'})

In [4]:
df.to_csv('test_origin.csv', index=False)

### Расчет признаков

#### Признак `events_ordinal_number`

In [5]:
# весь код расчета признака должен быть в этом методе
def calculate_events_number(df):
    
    df['events_ordinal_number'] = df.groupby('user_id').cumcount().add(1)
    
    return df

#### Признак `second_event_time`

In [6]:
# весь код расчета признака должен быть в этом методе
def calculate_second_event_time(df):
    
    
    df['second_event_time'] = df[['user_id']].merge(
                                  df.groupby('user_id').time.apply(
                                      lambda x: x.iloc[1] if x.size > 2 else np.nan),
                                  left_on='user_id', 
                                  right_index=True, 
                                  how='left',
                              ).time
        
    return df

#### Признак `loan_ordinal_number`

In [7]:
# весь код расчета признака должен быть в этом методе
def calculate_loan_number(df):

    df['loan_ordinal_number'] = df.groupby('user_id').is_loan.cumsum().add(1).sub(df.is_loan)

    return df

#### Признак `events_time_diff`

In [8]:
# весь код расчета признака должен быть в этом методе
def calculate_time_diff(df):

    df['events_time_diff'] = df.groupby('user_id').time.apply(lambda x: x - x.shift())

    return df

#### Признак `previous_loans_max_amount`

In [9]:
# весь код расчета признака должен быть в этом методе
def calculate_previous_max_amount(df):
    
    df['previous_loans_max_amount'] = df[['user_id']].merge(
                                          df[['user_id']].merge(
                                              df[df.is_loan ==1].amount, 
                                              left_index=True, 
                                              right_index=True, 
                                              how='left'
                                              ).groupby('user_id').amount.apply(lambda x: x.shift().expanding().max()
                                              ).reset_index(level=0, drop=True), 
                                          left_index=True, 
                                          right_index=True,
                                          how='left').amount
    
    return df

### Сохранение итогового датасета с признаками

Этот код трогать не нужно.

In [10]:
def calculate_feature(df):
    calculate_events_number(df)
    calculate_second_event_time(df)
    calculate_loan_number(df)
    calculate_time_diff(df)
    calculate_previous_max_amount(df)

In [11]:
%%timeit -o 
calculate_feature(df)

1.03 s ± 31.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 1.03 s ± 31.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [111]:
columns = ['time']
rez = _
df_time = pd.DataFrame([rez], None, columns)
df_time.to_csv('time.csv', index=False)

In [47]:
df.to_csv('test.csv', index=False)